In [14]:
import torch
import numpy as np
import scipy.sparse as ss
import scipy.io
import matplotlib.pyplot as plt
%matplotlib inline

In [145]:
class BCG():
    
    def __init__(self, A, b, prior_mean, prior_cov, eps, m_max):
        
        self.A = A
        self.b = b
        self.x0 = prior_mean
        self.sigma0 = prior_cov
        self.eps = eps
        self.max = m_max

    def bcg(self):
        
        sigmaF = [] #np.concatenate?
        
        A = self.A
        b = self.b
        x0 = self.x0
        sigma0 = self.sigma0
        eps = self.eps
        m_max = self.max
      #  print(torch.mm(A, x0).shape)
        r_m = b - torch.mm(A, x0)
        r_m_dot_r_m = torch.mm(r_m.t(), r_m)
        s_m = r_m
        x_m = x0
        
        nu_m = 0
        m = 0
        d = b.shape[0]
        
        while True:
            
            sigma_At_s = torch.mm(sigma0, torch.mm(A.t(), s_m))
            A_sigma_A_s = torch.mm(A, sigma_At_s)
            
            E_2 = torch.mm(s_m.t(), A_sigma_A_s)
            alpha_m = r_m_dot_r_m / E_2
            x_m += alpha_m * sigma_At_s
            r_m -= alpha_m * A_sigma_A_s
            nu_m += r_m_dot_r_m * r_m_dot_r_m / E_2
            sigma_m = ((d - 1 - m) * nu_m / (m + 1)).sqrt() ##??
            prev_r_m_dot_r_m = r_m_dot_r_m
            r_m_dot_r_m = torch.mm(r_m.t(), r_m)
            E = E_2.sqrt()
            sigmaF.append(sigma_At_s / E)
            
            m +=1
            
            beta_m = r_m_dot_r_m / prev_r_m_dot_r_m
            s_m = r_m + beta_m *s_m
            
            #add minimal no of iterations
            if sigma_m < eps:
               # print(E_2)
                break
            '''else sqrt(r_m_dot_r_m) < eps: - traditional residual-minimising strategy
                break'''
            if m == m_max or m == d:
                
                raise
                
        return x_m, sigmaF, nu_m/m

In [46]:
A = torch.tensor(np.array([[1, 0, 0], [0, 1, 0], [0, 0, 0.9]]), dtype=torch.double)
b = torch.tensor(np.array([[1, 4, 9]]).T, dtype=torch.double)

prior_mean = torch.tensor(np.array([[0, 0, 0]]).T, dtype=torch.double)
prior_cov = torch.tensor(np.linalg.inv(A))
eps = 1e-3
m_max = 10

bcg = BCG(A, b, prior_mean, prior_cov, eps, m_max)
bcg.bcg()

(tensor([[ 1.0000],
         [ 3.9999],
         [10.0000]], dtype=torch.float64), [tensor([[0.1055],
          [0.4219],
          [0.9492]], dtype=torch.float64), tensor([[-0.2224],
          [-0.8897],
          [ 0.4202]], dtype=torch.float64), tensor([[0.0864],
          [0.3455],
          [0.9850]], dtype=torch.float64)], tensor([[35.6665]], dtype=torch.float64))

In [193]:
A = scipy.io.loadmat('sparse_matrix.mat')['spmat']
x = np.random.normal(size=100)
b = A.dot(x)

In [194]:
A_dense = A.todense()

In [195]:
#prior_mean = torch.zeros([100])
prior_cov = torch.diag(torch.ones(100)).double()
A = torch.tensor(A_dense)


In [196]:
b = torch.tensor(b[np.newaxis, :].T, dtype=torch.double)

In [199]:
prior_mean = torch.tensor(np.zeros(100)[np.newaxis, :].T, dtype=torch.double)
#prior_cov = torch.tensor(np.linalg.inv(A))
eps = 1e-3
m_max = 100

bcg = BCG(A, b, prior_mean, prior_cov, eps, m_max)
res= bcg.bcg()

In [200]:
np.linalg.norm(np.array(res[0]) - x)

135.5385094862896